In [140]:
from bs4 import BeautifulSoup, NavigableString, Tag
import pandas as pd
import requests
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
import numpy as np
import glob
import time
import re

In [249]:
chrome_options = webdriver.ChromeOptions()
#save any files in the current working directory
#prefs = {'download.default_directory' : os.getcwd()}
#chrome_options.add_argument("--headless")
#chrome_options.add_experimental_option('prefs', prefs)
service = ChromeService(executable_path=ChromeDriverManager().install())

driver = webdriver.Chrome(service = service, options=chrome_options)

In [270]:
def getAllocationCategories(year, driver):
    
    #Navigate to the memorandums page for the given year
    url = 'https://infohub.nyced.org/reports/financial/financial-data-and-reports/school-allocation-memorandums'
    year = str(year)
    driver.get(url)
    link2memorandums = ''
    elements = driver.find_elements(By.CSS_SELECTOR, 'a[target="_blank"]')
    for element in elements:
        if year in element.text:
            link2memorandums = element.get_attribute('href')
            break
    driver.get(link2memorandums)
    
    #Go to the 	School Allocation Memorandums sorted numerically
    driver.find_element(By.XPATH,"//*[contains(text(), 'SAMs by Numbers')]").click()
    
    #Make a list of all the links to allocation memorandums and category titles
    url_prefix = link2memorandums.split('am_')[0]
    urls = []
    categories = []
    page = requests.get(driver.current_url)
    soup = BeautifulSoup(page.text, 'html.parser')
    table = soup.find('table')

    ran = False

    for row in table:
        try:
            #FSF has multiple entries but we only need the memorandum
            for a in row.find_all('a', href=True):
                if a.text == 'Fair Student Funding Memorandum':
                    urls.append(url_prefix + a['href'])
                    categories.append(a.text)
                #After 'Allocation Summary by District' marks the start of memorandum links
                if 'Allocation Summary by District' in a.text:
                    ran = True
                    continue
                #If you have passed 'Allocation Summary by District' start to save the category titles and links
                if ran:
                    if a['href'].startswith('http'):
                        urls.append(a['href'])
                    else:
                        urls.append(url_prefix + a['href'])
                    categories.append(a.text)
        except:
            continue
    
    #Formatting the titles
    for i, category_title in enumerate(categories):
        categories[i] = re.sub("\\r|\\t|\\n","",category_title)
    
    #List of dictionaries with allocation titles and galaxy listing
    categories_list = []

    #Add the galaxy listings for every allocation title
    for i, url in enumerate(urls):
        driver.get(url)
        funding_titles = []
        
        #Find elements with style attritbute and are bold and blue
        elements = driver.find_elements(By.CSS_SELECTOR, '[style]')
        for element in elements:
            if ('fontweightbold' in (re.sub(r'\W+', '',element.get_attribute("style")).lower())) and ('colorblue' in (re.sub(r'\W+', '',element.get_attribute("style")).lower()) or 'color0000FF' in (re.sub(r'\W+', '',element.get_attribute("style")).lower())):
                funding_titles.append(element.text)
                
        #Find elements with bold tag that are blue
        elements = driver.find_elements(By.TAG_NAME, 'b')
        for parentElement in elements:
            if (parentElement.find_elements(By.CSS_SELECTOR, '[color="#0000FF"]')):
                childElements = (parentElement.find_elements(By.CSS_SELECTOR, '[color="#0000FF"]'))
                for childElement in childElements:
                    funding_titles.append(childElement.text)
            elif (parentElement.find_elements(By.CSS_SELECTOR, '[style="color: blue"]')):
                childElements = (parentElement.find_elements(By.CSS_SELECTOR, '[style="color: blue"]'))
                for childElement in childElements:
                    funding_titles.append(childElement.text)
        
        #Find elements that are blue (first) and have a child element with the bold tag
        elements = driver.find_elements(By.CSS_SELECTOR, '[color="#0000FF"]')
        for parentElement in elements:
            if (parentElement.find_elements(By.TAG_NAME, 'b')):
                childElements = (parentElement.find_elements(By.TAG_NAME, 'b'))
                for childElement in childElements:
                    funding_titles.append(childElement.text)
        
        #Find elements in a table that are blue and bold
        elements = driver.find_elements(By.CSS_SELECTOR, 'tbody')
        for element in elements:
            if ('fontweightbold' in (re.sub(r'\W+', '',element.get_attribute("style")).lower())) and ('colorblue' in (re.sub(r'\W+', '',element.get_attribute("style")).lower()) or 'color0000FF' in (re.sub(r'\W+', '',element.get_attribute("style")).lower())):
                funding_titles.append(element.text)
        
        #Formatting for galaxy titles
        if '\n' in ''.join(funding_titles):
            temp = funding_titles.copy()
            for title in funding_titles:
                if '\n' in title:
                    temp.remove(title)
                    temp = temp + title.split('\n')
                    funding_titles = temp
        
        #Remove duplicates
        funding_titles = list(set(funding_titles))
        
        
        categories_list.append({"Category": categories[i], "Galaxy Titles": funding_titles})

            
    return categories_list
    

In [271]:
allocationCategories2021 = getAllocationCategories(2021, driver)
allocationCategories2021

[{'Category': 'Fair Student Funding Memorandum', 'Galaxy Titles': []},
 {'Category': 'Arts Supplemental Funding through Fair Student Funding',
  'Galaxy Titles': ['TL Fair Student Funding HS', 'TL Fair Student Funding']},
 {'Category': 'Career and Technical Education Supplement',
  'Galaxy Titles': ['TL STEAM HS', 'TL CTE Programs HS']},
 {'Category': 'ELA and Math Supplemental Support Services',
  'Galaxy Titles': ['TL ELA/Math Student Support',
   'Title IV Well Rounded ELA Math Support']},
 {'Category': 'Contract for Excellence',
  'Galaxy Titles': ['Contract for Excellence FY 09 HS',
   'TL CB School Staff',
   'Contract for Excellence FY 09']},
 {'Category': 'Collective Bargaining for School Based Staff',
  'Galaxy Titles': ['TL CB School Staff CW',
   'TL CB School Staff',
   'TL CB School Staff D79']},
 {'Category': 'Summer In the City',
  'Galaxy Titles': ['TL RS Student Summer in the City Services',
   'ESY Student Summer in the City Services RS']},
 {'Category': 'Title I Scho

In [268]:
allocationCategories2020 = getAllocationCategories(2020, driver)
allocationCategories2020

[{'Category': 'Fair Student Funding Memorandum', 'Galaxy Titles': []},
 {'Category': 'Deferred Program Planning Initiative (DPPI) ',
  'Galaxy Titles': ['TL Deferred Program Planning Initiative',
   'TL Deferred Program Planning Initiative (HS/CW/D79)',
   'TL Deferred Program Planning Initiative D79']},
 {'Category': 'Arts Supplemental Funding thru Fair Student Funding',
  'Galaxy Titles': ['TL Fair Student Funding HS', 'TL Fair Student Funding']},
 {'Category': 'ELA and Math Supplemental Support Services',
  'Galaxy Titles': ['TL ELA/Math Student Support']},
 {'Category': 'Contracts for Excellence (C4E) Discretionary Allocations to Schools',
  'Galaxy Titles': ['Contract for Excellence FY09 HS',
   'Contract for Excellence FY09']},
 {'Category': 'Collective Bargaining for School Based Staff',
  'Galaxy Titles': ['TL CB School Staff D88']},
 {'Category': 'Summer In the City', 'Galaxy Titles': []},
 {'Category': 'Title I School Allocations',
  'Galaxy Titles': ['TL CB School Staff']},


In [257]:
allocationCategories2022 = getAllocationCategories(2022, driver)
allocationCategories2022

[{'Category': 'Arts Supplemental Funding through Fair Student Funding',
  'Galaxy Titles': ['TL Fair Student Funding HS', 'TL Fair Student Funding']},
 {'Category': 'Career and Technical Education Supplement',
  'Galaxy Titles': ['TL STEAM HS',
   'TL CTE Programs HS',
   'CRRSA E&E Early College']},
 {'Category': 'ELA and Math Supplemental Support Services',
  'Galaxy Titles': ['TL ELA/Math Student Support',
   'Title IV Well Rounded ELA Math Support']},
 {'Category': 'Contract for Excellence',
  'Galaxy Titles': ['Contract for Excellence FY 09 HS',
   'TL CB School Staff',
   'Contract for Excellence FY 09']},
 {'Category': 'Collective Bargaining for School Based Staff',
  'Galaxy Titles': ['TL CB School Staff CW',
   'TL CB School Staff',
   'TL CB School Staff D79']},
 {'Category': 'Summer Rising',
  'Galaxy Titles': ['ARPA Academic Recovery AP Summer Rising',
   'ARPA',
   'TL Summer Rising Support Shared',
   'ARPA Academic Recovery Summer Rising',
   'ARPA, Title III, and Title 

In [208]:
district_5_allocations = pd.read_csv(r'C:\Users\ebroh\BetaNYC\School Budgets\School-Budget-and-Allocations\data\district 5\allocation_district_5.csv')
district_5_allocations

,Unnamed: 0,allocation_category,amount,location_code,fiscal_year
0,0,AIDP Attendance Shared,"$ 150,490",M692,2022
1,1,ARPA Academic Recovery,"$ 112,682",M692,2022
2,2,ARPA Academic Recovery Arts,"$ 28,171",M692,2022
3,3,ARPA Academic Recovery SE After-School Instruc...,"$ 16,242",M692,2022
4,4,ARPA Academic Recovery SE Counseling,"$ 2,223",M692,2022
...,...,...,...,...,...
6901,6901,TL Summer in the City Shared,"$ 12,969",M030,2018
6902,6902,TL Translation Services,"$ 1,027",M030,2018
6903,6903,TL Vision for School Improvement,"$ 19,097",M030,2018
6904,6904,Universal PreK Quality Full-Day,"$ 205,776",M030,2018


In [209]:
M125_allocations = district_5_allocations[district_5_allocations['location_code'] == 'M125']
M125_allocations

,Unnamed: 0,allocation_category,amount,location_code,fiscal_year
5282,5282,4K for All,"$ 475,301",M125,2022
5283,5283,ARPA Academic Recovery,"$ 79,229",M125,2022
5284,5284,ARPA Academic Recovery AP Summer Rising,"$ 17,990",M125,2022
5285,5285,ARPA Academic Recovery Arts,"$ 19,807",M125,2022
5286,5286,ARPA Academic Recovery SE After-School Instruc...,"$ 24,609",M125,2022
...,...,...,...,...,...
5517,5517,TL Talented and Gifted,$ 177,M125,2018
5518,5518,TL Terminal and Paid Leaves,"$ 51,566",M125,2018
5519,5519,TL Translation Services,$ 984,M125,2018
5520,5520,TL Vision for School Improvement,"$ 19,097",M125,2018


In [274]:
M125_2022 = M125_allocations[M125_allocations['fiscal_year'] == 2022]
M125_2022 = M125_2022.drop(columns=('Unnamed: 0'))
M125_allocation_categories = M125_2022['allocation_category'].unique()
all_2022_categories = []
for i in allocationCategories2022:
    all_2022_categories = all_2022_categories + i['Galaxy Titles']
overlap = [x for x in M125_allocation_categories if x not in all_2022_categories]
overlap

['ARPA LA Initial Register Relief',
 'ARPA LA MYA Register Relief',
 'IDEA RS IEP Para',
 'Rollover Smart Schools Bond Act',
 'Title IIA Supplemental',
 'TL FSF Post-IEP Support',
 'TL Funds Over Formula',
 'TL Itinerant Average Offset',
 'TL Itinerant Staff Shared',
 'TL NYSTL Library Books',
 'TL Parent Coordinator',
 'TL RS Speech Sixth Period Coverage',
 'TL Terminal and Paid Leaves']

In [278]:
M125_2021 = M125_allocations[M125_allocations['fiscal_year'] == 2021]
M125_2021 = M125_2021.drop(columns=('Unnamed: 0'))
M125_allocation_categories = M125_2021['allocation_category'].unique()
all_2021_categories = []
for i in allocationCategories2021:
    all_2021_categories = all_2021_categories + i['Galaxy Titles']
overlap = [x for x in M125_allocation_categories if x not in all_2021_categories]   
overlap

['4K for All',
 'Title I SWP Summer in the City',
 'Title IIA Supplemental',
 'TL Arts Studio',
 'TL Covid MY Register Loss Restoration',
 'TL FSF Post-IEP Support',
 'TL Funds Over Formula',
 'TL HEW School Wellness Council',
 'TL Implicit Bias Awareness',
 'TL Instructional Leadership Framework',
 'TL Periodic Assessment',
 'TL Permanent Placement Incentive',
 'TL RS Speech Sixth Period Coverage',
 'TL School to School Summer Transfers Shared']

In [279]:
M125_2020 = M125_allocations[M125_allocations['fiscal_year'] == 2020]
M125_2020 = M125_2020.drop(columns=('Unnamed: 0'))
M125_allocation_categories = M125_2020['allocation_category'].unique()
all_2020_categories = []
for i in allocationCategories2020:
    all_2021_categories = all_2021_categories + i['Galaxy Titles']
overlap = [x for x in M125_allocation_categories if x not in all_2021_categories]   
overlap

['FY20 Grant in Aid Assembly (08670)',
 'TL Arts Studio',
 'TL Funds Over Formula',
 'TL Instructional Leadership Framework',
 'TL MOSL',
 'TL Permanent Placement Incentive',
 'TL Potential Mid Year Adj to be Repaid',
 'TL Prior Year Encumbrance',
 'TL RS Speech Sixth Period Coverage',
 'TL Terminal and Paid Leaves']